In [25]:
# --- Instalación de librerías requeridas para análisis de datos, visualización y aprendizaje automático
!pip install --upgrade gspread pandas gspread-dataframe scikit-learn tqdm ipywidgets tensorflow matplotlib seaborn

# --- Autenticación de usuario en entorno Google Colab
from google.colab import auth
auth.authenticate_user()

# --- Importación de librerías esenciales
import random, gspread
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

# --- Utilidades para preprocesamiento y métricas de evaluación
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.utils import class_weight

# --- Integración con Google Sheets
from gspread_dataframe import set_with_dataframe
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# --- Configuración del entorno de visualización en Colab
from google.colab import output
output.enable_custom_widget_manager()
from google.auth import default

# --- Librerías para construcción de modelos con redes neuronales
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# --- Configuración de semillas para reproducibilidad
random.seed(42)
np.random.seed(42)

# --- Conexión a la hoja de cálculo de Google Sheets
spreadsheet_id = '18J8OHNlZRBNWMfA_hHnltZsrM2vRi8qji3dr20hAPEw'
link_google_sheets = f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/edit?usp=sharing"
creds, _ = default()
gc = gspread.authorize(creds)
sh = gc.open_by_key(spreadsheet_id)
worksheet = sh.sheet1

# --- Carga de datos desde la hoja activa
values = worksheet.get_all_values()
headers = values[0]
rows = values[1:]
df = pd.DataFrame(rows, columns=headers)

# --- Validación de estructura de datos: se esperan columnas numeradas del 1 al 98
esperadas = [f"{i}." for i in range(1, 99)]
preguntas_validas = df.columns[3:]
for col in preguntas_validas:
    if not any(col.strip().startswith(e) for e in esperadas):
        raise ValueError(f"Columna no reconocida en formato esperado: {col}")
if df.columns.duplicated().any():
    raise ValueError("Existen columnas duplicadas en la hoja de cálculo.")

# --- Conversión y validación de respuestas binarias (0 y 1)
for col in preguntas_validas:
    df[col] = pd.to_numeric(df[col], errors='coerce')
if df[preguntas_validas].isnull().any().any():
    raise ValueError("Existen valores faltantes o no numéricos.")
if not df[preguntas_validas].isin([0, 1]).all().all():
    raise ValueError("Las respuestas deben ser únicamente 0 o 1.")
df[preguntas_validas] = df[preguntas_validas].astype(int)

print("✅ Datos cargados y validados correctamente.")
# --- Definición de mapeos CHASIDE para áreas de interés y aptitud
mapeo_interes = {
    'C': [98, 12, 64, 53, 85, 1, 78, 20, 71, 91],
    'H': [9, 34, 80, 25, 95, 67, 41, 74, 56, 89],
    'A': [21, 45, 96, 57, 28, 11, 50, 3, 81, 36],
    'S': [33, 92, 70, 8, 87, 62, 23, 44, 16, 52],
    'I': [75, 6, 19, 38, 60, 27, 83, 54, 47, 97],
    'D': [84, 31, 48, 73, 5, 65, 14, 37, 58, 24],
    'E': [77, 42, 88, 17, 93, 32, 68, 49, 35, 61]
}
mapeo_aptitud = {
    'C': [15, 51, 2, 46],
    'H': [63, 30, 72, 86],
    'A': [22, 39, 76, 82],
    'S': [69, 40, 29, 4],
    'I': [26, 59, 90, 10],
    'D': [13, 66, 18, 43],
    'E': [94, 7, 79, 55]
}
area_completa = {
    'C': 'Administrativas y Contables',
    'H': 'Humanísticas y Sociales',
    'A': 'Artísticas',
    'S': 'Medicina y Ciencias de la Salud',
    'I': 'Enseñanzas Técnicas',
    'D': 'Defensa y Seguridad',
    'E': 'Ciencias Experimentales'
}
clases = list(area_completa.keys())

# --- Cálculo de puntajes por fila según las preguntas afirmadas
def calcular_puntajes(row, mapeo):
    puntajes = {key: 0 for key in mapeo}
    for letra, preguntas in mapeo.items():
        for pregunta in preguntas:
            col_match = [col for col in row.index if col.strip().startswith(f"{pregunta}.")]
            if col_match and row[col_match[0]] == 1:
                puntajes[letra] += 1
    return puntajes

# --- Preparación de matriz de características (X) y escalado de variables
X = df.iloc[:, 3:]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- Generación de etiquetas para interés y aptitud según área con mayor puntaje
y_interes, y_aptitud = [], []
for _, row in df.iterrows():
    puntajes_i = calcular_puntajes(row, mapeo_interes)
    puntajes_a = calcular_puntajes(row, mapeo_aptitud)
    y_interes.append(max(puntajes_i, key=puntajes_i.get))
    y_aptitud.append(max(puntajes_a, key=puntajes_a.get))

# --- Conversión de etiquetas a formato numérico para entrenamiento
label_map = {c: i for i, c in enumerate(clases)}
y_interes_num = np.array([label_map[val] for val in y_interes])
y_aptitud_num = np.array([label_map[val] for val in y_aptitud])

# --- Validación de balance de clases
def validar_balance(y_labels, nombre):
    conteo = Counter(y_labels)
    for clase, cantidad in conteo.items():
        if cantidad < 5:
            print(f"⚠ Clase con baja representación: {clases[clase]} ({cantidad}) en {nombre}")

validar_balance(y_interes_num, "interés")
validar_balance(y_aptitud_num, "aptitud")

# --- División de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_interes_train, y_interes_test = train_test_split(
    X_scaled, y_interes_num, test_size=0.2, stratify=y_interes_num, random_state=42
)
_, _, y_aptitud_train, y_aptitud_test = train_test_split(
    X_scaled, y_aptitud_num, test_size=0.2, stratify=y_aptitud_num, random_state=42
)

# --- Definición de arquitectura de red neuronal multicapa
def crear_modelo(input_dim, num_clases):
    model = Sequential([
        tf.keras.Input(shape=(input_dim,)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(num_clases, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# --- Cálculo de pesos de clase para compensar desbalance
pesos_interes = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_interes_train), y=y_interes_train)
pesos_aptitud = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_aptitud_train), y=y_aptitud_train)

# --- Inicialización y entrenamiento de modelos para interés y aptitud
modelo_interes = crear_modelo(X_train.shape[1], len(clases))
modelo_aptitud = crear_modelo(X_train.shape[1], len(clases))

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

modelo_interes.fit(X_train, y_interes_train, epochs=100, validation_split=0.2,
                   class_weight=dict(enumerate(pesos_interes)), callbacks=[early_stop], verbose=0)
modelo_aptitud.fit(X_train, y_aptitud_train, epochs=100, validation_split=0.2,
                   class_weight=dict(enumerate(pesos_aptitud)), callbacks=[early_stop], verbose=0)

# --- Predicción sobre conjunto de prueba
y_interes_pred = np.argmax(modelo_interes.predict(X_test), axis=1)
y_aptitud_pred = np.argmax(modelo_aptitud.predict(X_test), axis=1)

print("✅ Modelos de red neuronal entrenados exitosamente.")
# --- Función principal para comparación de modelos ANN, Regresión Logística y Árbol de Decisión
# --- Evaluación comparativa de modelos para INTERÉS y APTITUD con encabezados visuales
def comparar_modelos():
    clear_output()
    print("📊 COMPARACIÓN DE MODELOS: ANN - Regresión - Árbol de Decisión")

    short_names = [f"P{i+1}" for i in range(X.shape[1])]
    leyenda = pd.DataFrame({
        "Código": short_names,
        "Pregunta original": X.columns.tolist()
    })

    X_train_short = pd.DataFrame(X_train, columns=short_names)
    X_test_short = pd.DataFrame(X_test, columns=short_names)

    def evaluar_modelos(y_train, y_test, y_pred_ann, modelo_ann, titulo, color_hex):
        # --- Título general para secciones de interés o aptitud
        display(HTML(f"""
        <div style="padding: 20px; background-color:{color_hex}; border-radius:10px; margin-bottom:30px;">
            <h2 style="color:white; font-size:26px; text-align:center;">🔍 Evaluación de modelos - {titulo} CHASIDE</h2>
        </div>
        """))

        modelos = {
            "Red Neuronal Artificial": (modelo_ann, 'ANN'),
            "Regresión Logística": (LogisticRegression(max_iter=1000), 'LR'),
            "Árbol de Decisión": (DecisionTreeClassifier(max_depth=4), 'DT')
        }

        resultados = []

        for nombre, (modelo, tipo) in modelos.items():
            if tipo == 'ANN':
                y_pred = y_pred_ann
            else:
                modelo.fit(X_train_short, y_train)
                y_pred = modelo.predict(X_test_short)

            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
            recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
            precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)

            resultados.append({
                "Modelo": nombre,
                "Accuracy": acc,
                "F1-Score": f1,
                "Recall": recall,
                "Precision": precision
            })

            # --- Matriz de confusión
            plt.figure(figsize=(6, 5))
            cm = confusion_matrix(y_test, y_pred)
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
            plt.title(f"{titulo} - Matriz de Confusión - {nombre}", fontsize=18)
            plt.xlabel("Predicción")
            plt.ylabel("Real")
            plt.tight_layout()
            plt.show()
            display(HTML("<br>"))

        # --- Gráfico de barras con métricas
        df_resumen = pd.DataFrame(resultados).sort_values("Accuracy", ascending=False)
        df_melt = df_resumen.melt(id_vars='Modelo', var_name='Métrica', value_name='Valor')

        plt.figure(figsize=(10, 6))
        sns.barplot(data=df_melt, x='Modelo', y='Valor', hue='Métrica')
        plt.title(f"{titulo} - Comparación de Desempeño por Modelo", fontsize=20)
        plt.ylabel("Valor")
        plt.ylim(0, 1)
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        display(HTML("<br><hr><br>"))

        # --- Curvas ROC por modelo
        display(HTML(f"<h3 style='color:#008B8B; font-size:20px;'>📈 {titulo} - Curvas ROC</h3>"))
        y_bin = label_binarize(y_test, classes=np.arange(len(clases)))

        for nombre, (modelo, tipo) in modelos.items():
            if tipo == 'ANN':
                y_score = modelo.predict(X_test)
            elif hasattr(modelo, "predict_proba"):
                y_score = modelo.predict_proba(X_test_short)
            else:
                continue

            fpr, tpr, roc_auc = {}, {}, {}
            for i in range(len(clases)):
                fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], y_score[:, i])
                roc_auc[i] = auc(fpr[i], tpr[i])

            plt.figure(figsize=(10, 6))
            for i in range(len(clases)):
                plt.plot(fpr[i], tpr[i], lw=2, label=f"{clases[i]} (AUC={roc_auc[i]:.2f})")
            plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
            plt.title(f"{titulo} - Curva ROC - {nombre}", fontsize=18)
            plt.xlabel("Falsos Positivos")
            plt.ylabel("Verdaderos Positivos")
            plt.grid(True)
            plt.legend(loc="lower right")
            plt.tight_layout()
            plt.show()
            display(HTML("<br>"))

        # --- Visualización del árbol de decisión con etiquetas breves
        modelo_arbol = DecisionTreeClassifier(max_depth=4)
        modelo_arbol.fit(X_train_short, y_train)

        fig, ax = plt.subplots(figsize=(36, 18))
        plot_tree(modelo_arbol,
                  filled=True,
                  class_names=clases,
                  feature_names=short_names,
                  rounded=True,
                  fontsize=14,
                  precision=2,
                  ax=ax)
        plt.title(f"{titulo} - Árbol de Decisión (nombres abreviados)", fontsize=20, pad=20)
        plt.subplots_adjust(left=0.02, right=0.98, top=0.92, bottom=0.06)
        plt.show()

        display(HTML(f"<h4 style='margin-top:20px;'>🧾 Leyenda de columnas del árbol - {titulo}</h4>"))
        display(leyenda)
        display(HTML("<br><hr><br>"))

    # --- Evaluación para INTERÉS con encabezado celeste
    evaluar_modelos(y_interes_train, y_interes_test, y_interes_pred, modelo_interes, "INTERÉS", "#00BCD4")

    # --- Evaluación para APTITUD con encabezado verde
    evaluar_modelos(y_aptitud_train, y_aptitud_test, y_aptitud_pred, modelo_aptitud, "APTITUD", "#4CAF50")

    # --- Botones finales
    mostrar_botones_finales()

# --- Registro de función en Google Colab para permitir ejecución desde botón HTML
from google.colab import output
output.register_callback('notebook.compararModelos', lambda: comparar_modelos())

# --- Interfaz visual del menú principal con estilo responsivo y botón de acción personalizado
def mostrar_menu_principal():
    clear_output()
    display(HTML("""
    <div style="text-align:center; padding: 30px; background-color:#1e1e1e; color:white; border-radius:12px;">
        <h2 style="color:#00FFFF; font-size:26px; margin-bottom:10px;">SISTEMA DE TEST VOCACIONAL INTELIGENTE</h2>
        <h1 style="color:#DA70D6; font-size:30px; margin:0;">MENÚ PRINCIPAL</h1>
        <p style="font-size:18px; color:#4FC3F7; margin-top:10px;">
            Comparación de modelos de predicción para perfiles CHASIDE
        </p>
        <br>
        <button onclick="google.colab.kernel.invokeFunction('notebook.compararModelos', [], {})"
            style="
                background-color:#4B8DF8;
                color:white;
                padding:14px 28px;
                font-size:18px;
                font-weight:bold;
                border:none;
                border-radius:10px;
                box-shadow: 0 4px 8px rgba(0,0,0,0.3);
                cursor:pointer;
                transition: 0.3s;
            "
            onmouseover="this.style.backgroundColor='#3A6FD0'"
            onmouseout="this.style.backgroundColor='#4B8DF8'">
            📊 COMPARACIÓN DE MODELOS
        </button>
    </div>
    """))
# --- Función para mostrar resultados individuales con estilo HTML informativo
def mostrar_resultado(nombre, area_interes, area_aptitud):
    html = f"""
    <div style="background-color:#F0FFFF; padding:20px; margin-top:20px; border:1px solid #00CED1; border-radius:10px;">
        <h2 style="color:#1E90FF;">👤 {nombre}</h2>
        <h3 style="color:#20B2AA;">🏛️ Interés: {area_interes}</h3>
        <h3 style="color:#32CD32;">🎯 Aptitud: {area_aptitud}</h3>
    </div>
    """
    display(HTML(html))

# --- Función para exportar resultados al libro de Google Sheets
def guardar_resultados(resultados):
    df_resultados = pd.DataFrame(resultados)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_hoja = f"Resultados_TF_{timestamp}"
    hoja_nueva = sh.add_worksheet(title=nombre_hoja, rows="100", cols="30")
    set_with_dataframe(hoja_nueva, df_resultados)
    print(f"✅ Resultados guardados en la hoja: {nombre_hoja}")

# --- Menú de retorno y acceso directo a los resultados en Google Sheets
def mostrar_botones_finales():
    boton_volver = widgets.Button(
        description="🔄 VOLVER AL MENÚ",
        button_style='info',
        layout=widgets.Layout(width='250px', height='50px')
    )
    boton_volver.on_click(lambda b: mostrar_menu_principal())

    boton_sheets_html = f"""
    <a href="{link_google_sheets}" target="_blank" style="text-decoration:none;">
        <button style="
            background-color:#4CAF50;
            color:white;
            padding:10px 20px;
            border:none;
            border-radius:5px;
            min-width:250px;
            height:50px;
            font-size:16px;
        ">📄 VER RESULTADOS EN SHEETS</button>
    </a>
    """
    display(HTML(f"<br>{boton_sheets_html}<br><br>"))
    display(boton_volver)

# --- Ejecutar menú principal al cargar el sistema
mostrar_menu_principal()
